### Imports

In [8]:
from dataclasses import dataclass
from pathlib import Path
from Phishing_Detector.utils import *
from Phishing_Detector.constants import *
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
os.chdir('../')
#os.chdir('../DS_Projects/Phishing-Domain-Detection/')
os.getcwd()

'c:\\DS_Projects\\Phishing-Domain-Detection'

### Config Entity

In [35]:
@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir : Path
    x_train_file_path : Path
    x_test_file_path : Path
    y_train_file_path : Path
    y_test_file_path : Path
    saved_model_dir_path : Path
    saved_model_file_path : Path
    param_hidden_layer_sizes: tuple
    param_max_iter: int
    param_activation: str
    param_solver: str

### Configuration

In [36]:
class ConfigurationManager:

    def __init__(
            self, 
            config_path = CONFIG_FILE_PATH,
            params_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])

    def get_model_training_config(self) -> ModelTrainingConfig:
        
        config = self.config.model_training
        params = self.params
        create_directories([config.root_dir])

        model_training_config = ModelTrainingConfig(
            root_dir = config.root_dir,
            x_train_file_path = config.x_train_file_path,
            x_test_file_path = config.x_test_file_path,
            y_train_file_path = config.y_train_file_path,
            y_test_file_path = config.y_test_file_path,
            saved_model_dir_path = config.saved_model_dir_path,
            saved_model_file_path = config.saved_model_file_path,
            param_hidden_layer_sizes = params.hidden_layer_sizes,
            param_max_iter = params.max_iter,
            param_activation = params.activation,
            param_solver = params.solver
        )

        return model_training_config



In [39]:
class ModelTraining:
    
    def __init__(self, config:ModelTrainingConfig):
        print(f"{'>'*10} Starting Model Training Stage! {'<'*10}")
        
        print(f"Getting Configuration for Model Training Stage...")
        self.config = config
        print(f"Configurations : {config}")

        print(f"Creating required directories...")
        create_directories([self.config.saved_model_dir_path])

    def load_train_data(self):
        x_train = pd.read_csv(Path(self.config.x_train_file_path))
        y_train = pd.read_csv(Path(self.config.y_train_file_path))

        return x_train, y_train

    def train_model(self):
        print(f"Loading training data...")
        x_train, y_train  = self.load_train_data()
        print(f"Training data loaded successfully...")

        print(f"Initializing the MLPClassifier model...")
        mlp_clf = MLPClassifier(
            hidden_layer_sizes= self.config.param_hidden_layer_sizes,
            max_iter= self.config.param_max_iter,
            activation= self.config.param_activation,
            solver= self.config.param_solver
        )
        print(f"Fitting the training data in the model...")
        mlp_clf.fit(x_train, y_train)
        print(f"Model Training Complete...")

        print(f"Saving the model at {self.config.saved_model_file_path}...")
        self.save_model(mlp_clf)
        print(f"Model saved at {self.config.saved_model_file_path}...")

    def save_model(self, model):
        with open(Path(self.config.saved_model_file_path), 'wb') as model_path:
            joblib.dump(model, model_path)




In [40]:
try:
    config = ConfigurationManager()
    model_training_config = config.get_model_training_config()
    model_trainer = ModelTraining(config = model_training_config)
    model_trainer.train_model()
except Exception as e:
    raise e

>>>>>>>>>> Starting Model Training Stage! <<<<<<<<<<
Getting Configuration for Model Training Stage...
Configurations : ModelTrainingConfig(root_dir='model_training', x_train_file_path='artifacts/data_transformation/train_test_data/x_train.csv', x_test_file_path='artifacts/data_transformation/train_test_data/x_test.csv', y_train_file_path='artifacts/data_transformation/train_test_data/y_train.csv', y_test_file_path='artifacts/data_transformation/train_test_data/y_test.csv', saved_model_dir_path='artifacts/model_training/saved_models', saved_model_file_path='artifacts/model_training/saved_models/model.joblib', param_hidden_layer_sizes=BoxList([150, 100, 50]), param_max_iter=300, param_activation='relu', param_solver='adam')
Creating required directories...
Loading training data...
Training data loaded successfully...
Initializing the MLPClassifier model...
Fitting the training data in the model...


c:\DS_Projects\Phishing-Domain-Detection\phish_predictor_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model Training Complete...
Saving the model at artifacts/model_training/saved_models/model.joblib...
Model saved at artifacts/model_training/saved_models/model.joblib...
